In [ ]:
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt

# Simulate the arrival
arr_rate = 2
srv_rate = 4
duration = 18000
t = 0
jobs_iat = []
while t <= duration:
    iat = np.random.exponential(1/arr_rate)
    t += iat
    jobs_iat.append(iat)

# Simulate the service
i = 0
waiting_times = []
response_times = []
queue = []
in_service = [0,0] #[service_start,end]
busy_times = []
queue_size = [[0,0]]
t = 0

for job in jobs_iat:
    t += jobs_iat[i]
    st = np.random.exponential(1/srv_rate)
    #print(t)
    if t > in_service[1] and len(queue) > 0: #server is empty, but there jobs waiting in queue
        #fifoing the existing queue retroactively
        while(t>in_service[1] and len(queue) > 0):
            popped = queue.pop(0)
            pop_st = popped[0]
            start_queue = popped[1]
            queue_size.append([in_service[1],len(queue)])
            waiting_times.append(in_service[1]-start_queue)
            print('Server is free and occupied again at '+ str(round(in_service[1],3)))
            print('Job (old) was served for '+str(round(in_service[1]-in_service[0],3)))
            in_service = [in_service[1], in_service[1]+pop_st]
            busy_times.append(in_service)
            response_times.append(in_service[1]-start_queue)
            print('Job (new) was waiting in queue for '+str(round(in_service[0]-start_queue,3)))
            print('Queue size is now '+str(len(queue)))
            print('---------')
    if t > in_service[1] and len(queue) == 0: #server is (still) empty and queue is also empty
        if in_service[0] > 0:
            print('Server is free at '+ str(round(in_service[1],3)))
            print('Job was served for '+str(round(in_service[1]-in_service[0],3)))
            print('Queue size is now '+str(len(queue)))
            print('---------')
        #get in service
        waiting_times.append(0)
        in_service = [t,t+st]
        busy_times.append(in_service)
        response_times.append(st)
        print('Job arrived at: '+str(round(t,3)))
        print('Server is occupied at '+ str(round(t,3)))
        print('Job did not wait in queue')
        print('---------')
    else: #server is busy, regardless of queue
        queue.append([st,t])
        queue_size.append([t,len(queue)])
        print('Job arrived at: '+str(round(t,3)))
        print('Job is put in queue')
        print('Queue size is now '+str(len(queue)))
        print('---------')
    i += 1

# Derive the idle times
idle_times = [[0,busy_times[0][0]]]
for i in range(1,len(busy_times)):
    if busy_times[i][0] != busy_times[i-1][1]:
        idle_times.append([busy_times[i-1][1],busy_times[i][0]])
        
# Derive server busyness jump times
jump_times = [[0,0]]

i = 1
j = 0
while i < len(idle_times) and j < len(busy_times):
    i = min(i,len(idle_times)-1)
    j = min(j,len(busy_times)-1)
    if idle_times[i][0] < busy_times[j][0]:
        if jump_times[-1][1] == 1:
            jump_times.append([idle_times[i][0],0])
        i+=1
    else:
        if jump_times[-1][1] == 0:
            jump_times.append([busy_times[j][0],1])
        j+=1

In [ ]:
print(mean(waiting_times))
print(mean(response_times))

In [ ]:
# Plot the server busyness and queue size
start_t = 0
end_t = 30

tmp_jump = []
tmp_queue = []
for j in jump_times:
    if j[0] < start_t:
        continue
    if j[0] > end_t:
        break
    tmp_jump.append(j)
tmp_jump.append([end_t,tmp_jump[-1][1]])

for q in queue_size:
    if q[0] < start_t:
        continue
    if q[0] > end_t:
        break
    tmp_queue.append(q)
tmp_queue.append([end_t,tmp_queue[-1][1]])

plt.figure(figsize=(17,5))
plt.step([q[0] for q in tmp_jump], [q[1] for q in tmp_jump], 'r', where='post', label = 'Server busy')
plt.step([q[0] for q in tmp_queue], [q[1] for q in tmp_queue], 'b', where='post', label = 'Queue size')
plt.legend()
plt.show()

# Plot the average response and waiting times
avg_wait = []
n = 1
tot = 0
for w in waiting_times:
    tot += w
    avg_wait.append(tot/n)
    n += 1

avg_resp = []
n = 1
tot = 0
for w in response_times:
    tot += w
    avg_resp.append(tot/n)
    n += 1

plt.figure(figsize=(17,10))
plt.plot([i for i in range(len(avg_wait))], avg_wait, label = "Avg Wait Times")
plt.plot([i for i in range(len(avg_resp))], avg_resp, label = "Avg Response Times")
plt.legend()
plt.show()